## ITensor and Tensor Network Diagrams
ITensor is a library for efficient tensor computations, particularly useful in quantum physics and numerical simulations. First, we look at tensor network diagrams.

*All diagrams sourced from [tensornetworks.org](https://tensornetwork.org/diagrams/)*

Let's quickly review some important notation: Einstein summation convention.

> *"I have made a great discovery in mathematics; I have suppressed the summation sign every time that the summation must be made over an index which occurs twice..."*

To simplify expressions involving tensors with many indices, we hide the summation symbol $\sum$ by *implying* that whenever an index variable appears twice in a single term, it implies summation over all the values of that index. For example, matrix multiplication becomes:
$$ C_{ik} = \sum_j A_{ij} B_{jk} \implies C_{ik} = A_{ij} B_{jk} $$

<div style="display: flex; justify-content: space-around;">
  <img src="figs/types.png" alt="Types" width="45%">
  <img src="figs/diagrams.png" alt="Diagrams" width="45%">
</div>

Let's implement the above diagrams using ITensor.

In [27]:
using ITensors

# vector v_j
j = Index(4, "j") # index ranges over 4 values, labelled "j"
v = randomITensor(j) # random ITensor in R^4 with index j

# matrix M_ij
i = Index(3, "i") # index ranges over 3 values, labelled "i"
M = randomITensor(i, j) # random ITensor in R^{3x4} with indices i,j

# 3-tensor T_ijk
k = Index(2, "k") # index ranges over 2 values, labelled "k"
T = randomITensor(i, j, k) # random ITensor in R^{3x4x2} with indices i,j,k

# implementing the diagram contractions

# mat-vec product u_i = M_ij v_j
u = M * v # ITensors automatically contract over shared indices

# mat-mat product C_ik = A_ij B_jk
A = randomITensor(i, j)
B = randomITensor(j, k)
C = A * B # contracts over index j

# trace of AB, Tr(AB) = A_ij B_ji
TrAB = A * B # contracts over index j
TrAB = sum(TrAB) # sum over remaining indices to get scalar

-3.24233830315135

<span style="color: blue;">Task:</span> Implement the following tensor contraction.
$$ \mathrm{Tr}(ABCD) = A_{ij} B_{jk} C_{kl} D_{li}, \quad A, B, C, D \in \mathbb{R}^{3 \times 3},\quad A_{ij}, B_{jk}, C_{kl}, D_{li} \sim \mathcal{N}(0,1) $$
Note that we are using Einstein summation convention here. All indices repeat, implying everything is summed over and the result is a scalar (as a trace should be).

<details>
  <summary> <strong> code </strong> </summary>

```julia
# redefine i, j, k, l
i = Index(3, "i")
j = Index(3, "j")
k = Index(3, "k")
l = Index(3, "l")

# define random tensors A, B, C, D
A = randomITensor(i, j)
B = randomITensor(j, k)
C = randomITensor(k, l)
D = randomITensor(l, i)

# compute the trace of ABCD
TrABCD = A * B * C * D

# sum over all indices to get a scalar
TrABCD = sum(TrABCD)

println("Trace of ABCD:")
println(TrABCD)
```

</details>

<details>
  <summary>tensor diagram</summary>

<div style="text-align: center;">
    <img src="figs/trabcd.jpeg" width="500"/>
</div>
  <!-- <img src="figs/trabcd.jpeg" width="500", style="centered"/> -->
</details>


In [28]:
# TODO: try implementing Tr(ABCD) here!

# good contraction checks, @show inds
@show inds(u) # should show index i
@show inds(C) # should show indices i,k

inds(u) = ((dim=3|id=455|"i"),)
inds(C) = ((dim=3|id=455|"i"), (dim=2|id=79|"k"))


((dim=3|id=455|"i"), (dim=2|id=79|"k"))

Now, let's use tensor computations for quantum systems.

*Curse of Dimensionality.* A chain of $N$ spin-$1/2$ particles has a state space dimension of $\mathcal H \cong \mathbb C^{2^N}$. A many-body state in this space is represented using $2^N$ complex numbers. $$ \psi_{s_1, s_2, \ldots, s_N} = \sum_{s_1, \ldots, s_N \in \{0,1\}} c_{s_1, s_2, \ldots, s_N} |s_1\rangle \otimes |s_2\rangle \otimes \cdots \otimes |s_N\rangle $$


For $N = 10$, storing a general state vector requires $2^{10} = 1024$ complex numbers, which is manageable.

For $N=50$, storing a general state vector requires $2^{50} \approx 1.13 \times 10^{15}$ complex numbers, which is about 9 petabytes of memory!

This seems pretty hopeless beyond a few dozen sites. Tensor-networks methods tame this explosion by factorizing $\psi$ into smaller tensors and keeping *relevant* correlations.

$$ \psi_{s_1, s_2, \ldots, s_N} = \sum_{s_1, \ldots, s_N \in \{0,1\}} c_{s_1, s_2, \ldots, s_N} |s_1\rangle \otimes |s_2\rangle \otimes \cdots \otimes |s_N\rangle \quad \rightarrow \quad \psi_{s_1, s_2, \ldots, s_N} = A^{[1]}_{s_1} A^{[2]}_{s_2} \cdots A^{[N]}_{s_N} $$

This reduced the storage scaling from $O(d^N)$ to $O(N d m^2)$, where $m$ is the bond dimension controlling the amount of entanglement/correlation captured.

<div style="display: flex; justify-content: space-around;">
  <img src="figs/mps.png" alt="MPS" width="50%">
  <img src="figs/mpsalgo.png" alt="MPS Algorithm" width="50%">
</div>

*This algorithm is also called the tensor-train singular value decomposition (TT-SVD). Look [here](https://tensornetwork.org/mps/) for a visual introduction and Miles Stoudenmire's [notes](https://https://itensor.org/miles/)*.